In [1]:
import sys
sys.path.append('/public/src')
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

from factor_evaluation_server import FactorEvaluation, DataService
ds = DataService()
df = ds['ETHUSDT_15m_2020_2025']['2021-10-01':]

In [2]:
path="/public/data/factor_data/ETH_15m_factor_data.txt"
factors=pd.read_csv(path, sep='|')
factors.head()

,Unnamed: 0,ret_bollinger_rsi_signals,c_chu016,c_chu009,c_chu011,ret_hv_ratio_signals,ret_vao_signals,c_chu028,ret_ma120_cci_cross_sig_price,ret_ma20_volume_cross_signals,...,c_chu059,ret_mfi_sig_price,c_hide_009,c_chu044,c_hide_014,ret_rsi_ma120_cross_sig_price,c_hide_006,c_chu037,c_chu004,c_hide_010
0,2019-12-31 16:00:00,0,NaN,NaN,-0.0235,0,0,NaN,0,0,...,NaN,0,0.0000,7111059.2053,0.0000,0,0.0000,NaN,NaN,0.0000
1,2019-12-31 16:15:00,0,NaN,-0.0000,0.0650,0,0,NaN,0,0,...,NaN,0,0.0000,2926088.3661,0.0000,0,0.0000,0.0000,NaN,0.0000
2,2019-12-31 16:30:00,0,NaN,-0.0001,0.3217,0,0,NaN,0,0,...,NaN,0,0.0000,2166516.5598,0.0000,0,0.0000,0.0001,NaN,0.0000
3,2019-12-31 16:45:00,0,NaN,0.0000,0.1571,0,0,NaN,0,0,...,NaN,0,0.0000,135422.1719,0.0027,0,0.0000,0.0000,NaN,0.0000
4,2019-12-31 17:00:00,0,NaN,-0.0001,-0.0850,0,0,NaN,0,0,...,NaN,0,0.0000,12405.6168,0.0008,0,0.0000,0.0001,NaN,0.0000


In [3]:
for i in list(factors.columns):
    print(i)

Unnamed: 0
ret_bollinger_rsi_signals
c_chu016
c_chu009
c_chu011
ret_hv_ratio_signals
ret_vao_signals
c_chu028
ret_ma120_cci_cross_sig_price
ret_ma20_volume_cross_signals
c_chu026
c_chu015
c_chu029
c_chu012
c_chu058
c_hide_028
c_chu045
ret_ma_bbi_rsi_sig_price
c_chu023
c_hide_001
c_hide_032
c_chu019
c_chu032
c_chu049
c_chu051
c_hide_008
ret_macd_sig_price
c_chu022
ret_ma_arrangement_sig
ret_ma120_bolling_cross_sig_price
c_hide_019
c_hide_026
c_chu033
c_hide_030
ret_skdj_sig_price
ret_ma120_bbi_signals
c_chu043
ret_williams_r_sig_price
c_chu042
ret_ma20_rsi_macd_cross_sig_price
c_chu053
c_chu005
c_chu052
ret_cci_fibonacci_signals
ret_ma_cci_sig
c_chu021
c_hide_005
ret_ao_signals
ret_ma_atr_cross_sig_price
c_chu006
c_hide_021
c_hide_029
ret_rma_cross_sig_price
c_hide_002
ret_rsi_bb_ma_signal
c_chu010
c_chu048
c_chu041
c_hide_016
ret_kc_strategy
c_chu047
c_chu056
ret_ma120_macd_1_cross_sig_price
c_chu025
c_hide_011
c_chu018
c_hide_018
c_hide_031
c_chu054
ret_td_signals
c_chu001
c_chu040
c_

In [4]:
factors.index

RangeIndex(start=0, stop=192421, step=1)

In [5]:
def factor(df):
    """
    改进点：
    1. 结合成交量分布和价格位置双重确认
    2. 增加交易意图分析（主动买入/卖出）
    3. 使用动态时间窗口
    """
    # 动态窗口：根据波动率调整回溯期
    volatility = df['close'].pct_change().rolling(96).std()
    
    # 处理NaN和inf问题 - 填充缺失值，避免除以零
    volatility = volatility.fillna(volatility.mean())  # 填充NaN为均值
    volatility = volatility.replace(0, 1e-6)           # 避免除以零
    volatility = np.clip(volatility, 1e-6, None)        # 确保最小正值
    
    # 计算窗口大小并确保为整数
    window_size = (960 / (volatility * 100 + 1))
    window_size = np.nan_to_num(window_size, nan=480)   # 替换NaN为默认值
    window_size = np.clip(window_size, 480, 1440).astype(int)
    
    # 预计算关键指标
    avg_price = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ret = (df['close'] - df['open']) / df['open']
    abs_ret = np.abs(ret)
    buy_power = df['taker_buy_volume'] / df['volume']
    
    # 核心指标：量价确认得分
    # 结合价格变动方向、主动买入比例和交易规模
    S = np.sign(ret) * abs_ret * buy_power * np.log1p(df['volume'])
    
    # 初始化因子值数组
    factor_values = np.full(len(df), np.nan)
    
    # 找到有效的起始索引（确保有足够数据）
    start_idx = max(np.max(window_size), 96) + 1
    
    for i in range(start_idx, len(df)):
        w_size = window_size[i]
        start_idx = i - w_size
        end_idx = i - 1
        
        # 提取窗口数据
        window_avg_price = avg_price.iloc[start_idx:end_idx].values
        window_volume = df['volume'].iloc[start_idx:end_idx].values
        window_S = S.iloc[start_idx:end_idx].values
        
        # 按S值降序排序（聪明钱交易在前）
        sorted_idx = np.argsort(-window_S)
        sorted_volume = window_volume[sorted_idx]
        sorted_avg_price = window_avg_price[sorted_idx]
        
        # 动态阈值（基于波动率）
        threshold_ratio = 0.3 - 0.15 * (volatility[i] / volatility.max())  # 波动大时降低阈值
        threshold = np.sum(window_volume) * threshold_ratio
        
        # 识别聪明钱交易
        cum_volume = np.cumsum(sorted_volume)
        smart_mask = cum_volume <= threshold
        
        if np.any(smart_mask):
            # 计算聪明钱VWAP（加权平均价）
            smart_vwap = np.sum(sorted_avg_price[smart_mask] * 
                              sorted_volume[smart_mask]) / np.sum(sorted_volume[smart_mask])
            
            # 计算整体VWAP
            all_vwap = np.sum(window_avg_price * window_volume) / np.sum(window_volume)
            
            # 添加市场状态调整
            if all_vwap > window_avg_price[-96:].mean():  # 近期上涨
                factor_values[i] = smart_vwap / all_vwap
            else:  # 近期下跌
                factor_values[i] = 2 - (smart_vwap / all_vwap)  # 反转因子方向
    
    return pd.Series(-factor_values, index=df.index)

In [6]:
sig=factor(df)

In [7]:
# sig的index修改为arrange
sig = sig.reset_index(drop=True)

In [8]:
factors['sig']=sig

In [9]:
corr_matrix=factors.corr()

In [10]:
corr_matrix.iloc[-1,:]

# 我想看到这一行的全部数值
corr_matrix.iloc[-1,:].sort_values(ascending=False)

sig                  1.0000
open_time            0.0221
close_time           0.0221
c_chu059             0.0108
c_chu050             0.0107
                      ...  
c_chu030            -0.0236
c_hide_032          -0.0237
c_chu019            -0.0289
c_chu045            -0.0304
ret_stc_sig_price       NaN
Name: sig, Length: 142, dtype: float64